In [26]:
import numpy as np
from sklearn.cluster import MiniBatchKMeans
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from nba_api.stats.endpoints import leaguegamefinder
import pandas as pd

seasons = ["2023-24", "2024-25"]
dfs = []

for season in seasons:
    gamefinder = leaguegamefinder.LeagueGameFinder(
        season_nullable=season,
        season_type_nullable="Regular Season"
    )
    df = gamefinder.get_data_frames()[0]

    dfs.append(df)

league_games = pd.concat(dfs, ignore_index = True)

#No Duplicates
league_games = league_games.drop_duplicates(subset=["GAME_ID"])
DroppedNAN = league_games.dropna(subset=['PTS', 'AST', 'REB', 'TOV', 'PLUS_MINUS', 'FG_PCT', 'FT_PCT', 'FG3_PCT', 'WL'])


DroppedNAN.to_csv("NBA_Dataset.csv", index = False)

In [27]:
uiHomeTeam= input ("Select the 1st NBA team:")
uiAwayTeam= input ("Select the 2nd NBA team:")
df= pd.read_csv("NBA_Dataset.csv")
df = pd.DataFrame(df)
df["WL"]= df["WL"].map({'W': 1, 'L': 0})

X= df[["PTS","REB","AST","TOV","PLUS_MINUS","FG_PCT","FG3_PCT","FT_PCT"]]
Y= df[["WL"]]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

k = 6  # win vs loss style clusters
mbk = MiniBatchKMeans(
    n_clusters=k,
    batch_size=256,
    random_state=42
)

clusters = mbk.fit_predict(X_scaled)
df["Cluster"] = clusters
df.head()

cluster_win_prob = df.groupby("Cluster")["WL"].mean()

In [28]:
FEATURES = [
    "PTS", "REB", "AST", "TOV",
    "PLUS_MINUS", "FG_PCT", "FG3_PCT", "FT_PCT"
]
team_averages = (
    df.groupby("TEAM_NAME")[FEATURES]
    .mean()
)

In [ ]:
from scipy.special import softmax

def get_last_n_games(team_name, n=5):
    team_games = df[df["TEAM_NAME"] == team_name].tail(n)
    return team_games[FEATURES].mean().values

def raw_win_probability(stats):
    scaled = scaler.transform([stats])
    distances = mbk.transform(scaled)[0]
    weights = softmax(-distances)
    return np.dot(weights, cluster_win_prob.values)

def normalize_probs(pA, pB):
    total = pA + pB
    return pA / total, pB / total


def predict_future_game(teamA_name, teamB_name, n=5):
    teamA_stats = get_last_n_games(teamA_name, n)
    teamB_stats = get_last_n_games(teamB_name, n)

    rawA = raw_win_probability(teamA_stats)
    rawB = raw_win_probability(teamB_stats)

    probA, probB = normalize_probs(rawA, rawB)

    winner = teamA_name if probA > probB else teamB_name

    print("🏀 Future Game Prediction")
    print("Predicted Winner:", winner)
    print(f"{teamA_name} Win Probability: {probA:.2%}")
    print(f"{teamB_name} Win Probability: {probB:.2%}")

    return winner, probA, probB

In [36]:
predict_future_game(uiHomeTeam, uiAwayTeam)
cluster_label_map = df.groupby("Cluster")["WL"].mean().apply(lambda x: 1 if x >= 0.5 else 0)
df["Predicted_WL"] = df["Cluster"].map(cluster_label_map)

accuracy = (df["Predicted_WL"] == df["WL"]).mean()
print(f"Model Accuracy: {accuracy:.2%}")


🏀 Future Game Prediction
Predicted Winner: New York Knicks
New York Knicks Win Probability: 63.36%
San Antonio Spurs Win Probability: 36.64%
Model Accuracy: 76.91%


C:\Users\denni\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\denni\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [37]:
from sklearn.metrics import log_loss, roc_auc_score, accuracy_score, brier_score_loss

def eval_set(name, model, Xs, ys):
    p = model.predict_proba(Xs)[:, 1]
    pred = (p >= 0.5).astype(int)

    print(f"\n{name}")
    print("  logloss :", log_loss(ys, p))
    print("  brier   :", brier_score_loss(ys, p))
    print("  auc     :", roc_auc_score(ys, p))
    print("  acc     :", accuracy_score(ys, pred))

eval_set("VAL", logreg, X_train, y_train)
eval_set("TEST", logreg, X_test, y_test)


NameError: name 'logreg' is not defined